<div style="text-align: right"><b> Ana Valentina López Chacón </b></div>
<div style="text-align: right"><b> Traducción Automática, 2024 - 2025 </b></div>

# **Portugués a Ingles: Finetuning con M2M100**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# path = '/content/drive/MyDrive/Colab Notebooks/TA/A3/'

Mounted at /content/drive


In [2]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q datasets jiwer tqdm pandas
!pip install -q torch torchvision torchaudio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip install -q evaluate
!pip install -q transformers==4.47.0
!pip install -q accelerate==0.27.0
!pip install -q peft
!pip install -q bitsandbytes==0.45.0
!pip install -q sacrebleu
!pip install -q unbabel-comet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 40.3 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

data_train = load_dataset("csv", data_files='/content/drive/MyDrive/Colab Notebooks/TA/A3/pt_en_ASR_Whisper_Medium_train.csv')
data_train = data_train.select_columns(["transcription", "translation"])
data_train = data_train["train"].train_test_split(test_size = 0.83, shuffle = False)

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
data_train["train"][:5]["transcription"]

[' altere o dispositivo de saída de áudio.',
 ' Até a data limite prevista no tratado, vários mices foram destruídos.',
 ' Eu balancei a cabeça.',
 ' Adsoni de Cranberries Zombie Aplicia de clássicas',
 ' Um cinegrafista filmando um jogo de basquete.']

In [6]:
data_train["train"][:5]["translation"]

['Change the audio output device.',
 'By the deadline shown in the treaty, various missiles had been destroyed.',
 'I shook my head.',
 'Add The Cranberries Zombie to the classics playlist.',
 'A cameraman filming a basketball game.']

In [7]:
data_train["test"][:5]["transcription"]

[' O repórter perguntou três vezes seguidas.',
 ' De mais.',
 ' Ela não respondeu nada.',
 ' De qualquer forma, você tem muitas coisas para fazer.',
 ' Aquele que paga por último nunca paga duas vezes.']

In [8]:
data_train["test"][:5]["translation"]

['The reporter asked three times in a row.',
 'Enough!',
 'She didn’t answer anything.',
 'In any case, you have a lot of stuff to do.',
 'The one who pays last never pays twice.']

In [9]:
max_tok_length = 275

from transformers import AutoTokenizer

checkpoint = "facebook/m2m100_418M"
src_code = "pt"
tgt_code = "en"

tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    padding=True,
    pad_to_multiple_of=8,
    src_lang=src_code,
    tgt_lang=tgt_code,
    truncation=False,
    max_length=max_tok_length,
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [10]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["transcription"],
        text_target = sample["translation"],
        )
    return model_inputs


In [11]:
tokenized_data_train = data_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/340 [00:00<?, ? examples/s]

Map:   0%|          | 0/1660 [00:00<?, ? examples/s]

In [12]:
dic = {}
for sample in tokenized_data_train['train']:
    sample_length = len(sample['input_ids'])
    if sample_length not in dic:
        dic[sample_length] = 1
    else:
        dic[sample_length] += 1

for i in range(1, max_tok_length + 1):
    if i in dic:
        print(f"{i:>2} {dic[i]:>3}")

 4   2
 5   4
 6  10
 7  22
 8  23
 9  29
10  37
11  32
12  23
13  31
14  23
15  19
16  25
17  12
18   7
19   5
20  11
21   6
22   7
23   7
24   2
25   1
26   1
28   1


In [13]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [14]:
from transformers import M2M100ForConditionalGeneration

model = M2M100ForConditionalGeneration.from_pretrained(
    checkpoint,
    quantization_config=quantization_config
    )


`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [15]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False, gradient_checkpointing_kwargs={'use_reentrant':False})

In [16]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

In [17]:
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 486,264,832 || trainable%: 0.4852


In [18]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=lora_model,
    pad_to_multiple_of=8
    )

In [19]:
from evaluate import load

metric = load("sacrebleu")

In [20]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    for i in range(len(labels)):
        labels[i] = [tokenizer.pad_token_id if j<0 else j for j in labels[i]]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### **Training**

In [21]:
from transformers import Seq2SeqTrainingArguments

batch_size = 2
model_name = checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir="./",
    run_name="finetuning2_es_to_en_lora",
    eval_strategy = "epoch",
    learning_rate=1e-4,
    report_to="wandb",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True
    )

In [22]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    lora_model,
    args,
    train_dataset=tokenized_data_train['train'],
    eval_dataset=tokenized_data_train['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [23]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.027571,45.619100,13.024700
2,No log,1.008080,46.114600,13.004800
3,1.103100,1.018072,46.096200,12.997000
4,1.103100,1.017774,46.044300,12.994000
5,1.103100,1.019747,46.016100,13.003000


Se han truncado las últimas 5000 líneas del flujo de salida.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class in

TrainOutput(global_step=850, training_loss=1.0331243896484374, metrics={'train_runtime': 4155.1034, 'train_samples_per_second': 0.409, 'train_steps_per_second': 0.205, 'total_flos': 66981941084160.0, 'train_loss': 1.0331243896484374, 'epoch': 5.0})

In [24]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(
    checkpoint,
)

print(generation_config)

GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1
}



### **Load Test Set**

In [25]:
from datasets import load_dataset

data_test = load_dataset("csv", data_files='/content/drive/MyDrive/Colab Notebooks/TA/A3/pt_en_ASR_Whisper_Medium_test.csv')

data_test

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'transcription', 'sentence', 'translation', 'transcription_clean', 'sentence_clean', 'translation_clean'],
        num_rows: 1000
    })
})

In [26]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["transcription"],
        text_target = sample["translation"],
        )
    return model_inputs

In [27]:
tokenized_data_test = data_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [28]:
test_batch_size = 32
batch_tokenized_test = tokenized_data_test['train'].batch(test_batch_size)

Batching examples:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [29]:
number_of_batches = len(batch_tokenized_test["transcription"])
output_sequences = []
for i in range(number_of_batches):
    inputs = tokenizer(batch_tokenized_test["transcription"][i], max_length=max_tok_length, truncation=False, return_tensors="pt", padding=True)
    output_batch = lora_model.generate(generation_config=generation_config,
                                       input_ids=inputs["input_ids"].cuda(),
                                       attention_mask=inputs["attention_mask"].cuda(),
                                       forced_bos_token_id=tokenizer.get_lang_id(tgt_code),
                                       max_length = max_tok_length, num_beams=1, do_sample=False,)
    output_sequences.extend(output_batch.cpu())

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [30]:
decoded_preds = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [31]:
references = tokenizer.batch_decode(tokenized_data_test["train"]["labels"], skip_special_tokens=True)

In [32]:
references[:5]

['Borrow money from people in the village',
 'Lock them up',
 'Youtube is still the best video platform.',
 'A girl and a boy kissing in the shadows',
 'I actually prefer to eat my sliced couscous as it is, instead of grilled.']

In [33]:
decoded_preds[:5]

['And money borrowed to the people of the village.',
 'We silence.',
 'YouTube is still the best video platform.',
 'Girl and boy kissing in the shadows.',
 'Specifically, my cut couscous I like eating well instead of grayed.']

In [34]:
from whisper.normalizers.basic import BasicTextNormalizer

normalizer = BasicTextNormalizer()

decoded_preds_clean = [normalizer(text) for text in decoded_preds]
references_clean = [normalizer(text) for text in references]

In [35]:
from evaluate import load

metric = load("sacrebleu")

In [36]:
result = metric.compute(predictions=decoded_preds_clean, references=references_clean)
print(f'BLEU score: {result["score"]:.1f}')

BLEU score: 44.8


In [37]:
from evaluate import load
comet_metric = load('comet')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [38]:
comet_score = comet_metric.compute(predictions=decoded_preds_clean, references=references_clean, sources=data_test["train"]["transcription"])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [39]:
print(f"COMET: {comet_score['mean_score'] * 100:.2f} %")

COMET: 83.05 %
